<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/Hello_cu_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello cu_cat: Easy automatic feature engineering for messy dataframes on GPUs!
Daniel, Tanmoy, Alex, Leo


[  [LOOM](https://www.loom.com/share/d7fd4980b31949b7b840b230937a636f?sid=6d56b82e-9f50-4059-af9f-bfdc32cd3509)] – press play and dive into analysis!

The latest [PyGraphistry[AI]](https://github.com/graphistry/pygraphistry/) release GPU accelerates its automatic feature encoding pipeline, by introducing a new member to the open source GPU dataframe ecosystem: cu_cat.

Cu_cat is our GPU-accelerated open source fork of the popular CPU Python  library dirty_cat.   Like dirty_cat, cu_cat makes it easy to convert messy dataframes filled with numbers, strings, and timestamps into numeric feature columns optimized for AI models. It adds interoperability for GPU dataframes and replaces key kernels and algorithms with faster and more scalable GPU variants. **Even on low-end GPUs, we are now able to tackle much larger datasets in the same amount of time – or for the first time! – with end-to-end pipelines. We typically save time with 3-5X speedups and will even see 10X+.**

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==23.06.00 cudf-cu11==23.06.00 cugraph-cu11==23.06.00 pylibraft_cu11==23.06.00 raft_dask_cu11==23.06.00 dask_cudf_cu11==23.06.00 pylibcugraph_cu11==23.06.00 pylibraft_cu11==23.06.00


In [ ]:
import cuml,cudf,os
cuml.__version__

'23.06.00'

In [ ]:
!pip install git+https://github.com/graphistry/pygraphistry.git@feat/gpu-featurization
### !pip install -U git+https://github.com/graphistry/cu-cat.git@DT6
!pip install -e git+https://github.com/dcolinmorgan/cu-cat.git@master#egg=cu-cat

# !git clone https://github.com/dcolinmorgan/cu-cat.git
# os.getcwd()

os.chdir('src/cu-cat')
import cu_cat
os.chdir('../..')

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import cudf
from time import time
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 200)

In [ ]:
import graphistry

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__


'0.29.5+84.gf0eb1bf'

In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv
# !pip install nvsmi
# import nvsmi
# gpu=str(nvsmi.get_gpu_processes()).split('|')[3].split(':')[1]

name
Tesla T4


In [ ]:
X=1
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
from time import time


## look at dataset (select non-sparse columns)

In [ ]:
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogsA=pd.read_parquet('part.88.parquet')


In [ ]:
winlogs=winlogsA[['LogonID','UserName','LogHost','Time','DomainName']]#,'LogonType','SubjectLogonID','Status','Destination','ServiceName']]#,i]]
winlogs=winlogs.sample(10000,replace=False)#.dropna()

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Graphistry[AI] - Feature Engineering:
## CUDA_cat GPU engine
- data: **5million row** x dozens of sparse columns Windows log dataset
- data subset: parred down to **20k** rows, selecting just dense columns

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
winlogs=winlogsA[['LogonID','UserName','LogHost','Time','DomainName']]#,'LogonType','SubjectLogonID','Status','Destination','ServiceName']]#,i]]
winlogs=winlogs.sample(10000,replace=False)

In [ ]:
g1 = graphistry.nodes(winlogs)
t=time()
g11=g1.umap(feature_engine='cu_cat',engine='cuml',memoize=False)
t2=time()-t
print(t2)

g11.plot()

53.1531982421875


## run loops per size and feature_engine, ie cu_cat and dirty_cat, remembering to set memoize=False

In [ ]:
bench=pd.DataFrame(columns=['dataset','row','col','cc_time','dc_time'])
# winlogsA=pd.read_parquet('part.88.parquet')
winlogsA=winlogsA[winlogsA.columns.drop(['ProcessID','LogonID','Source','ParentProcessID']).tolist()]
s=winlogsA.select_dtypes(include='int').columns
winlogsA[s] = winlogsA[s].astype("float")
# winlogsA.fillna(0,inplace=True)
# winlogsA.Time=pd.to_datetime(winlogsA.Time.astype(str).str[:6],errors='coerce')

In [ ]:

n=[2000,5000,10000,50000,500000]

for ii,i in enumerate(n):
    winlogs=winlogsA.sample(i,replace=False)#.fillna(0)

    g0 = graphistry.nodes((winlogs))
    t=time()
    # try:
    g00=g0.umap(feature_engine='cu_cat',engine='cuml',memoize=False,min_dist=0)
    t1=time()-t
    print('\n',t1,'\n')
    # except:
      # print('didnt run:',i,'/n')

    g1 = graphistry.nodes(winlogs)
    t=time()
    g11=g1.umap(feature_engine='dirty_cat',engine='cuml',memoize=False,min_dist=0)
    t2=time()-t
    print('\n',t2,'\n')
    bench.loc[1+bench.shape[0]]=['winlogs',winlogs.shape[0],winlogs.shape[1],t1,t2]
    bench.to_csv('winlogs_'+str(gpu.strip())+'_wide_bench_'+timestr+'.txt',sep='\t',mode='a')


In [ ]:
g00.addStyle(bg={'color': '#eee'}.plot()

#plot results and export

In [ ]:
bench=bench.rename(columns={'cc_time':'cu_cat','dc_time':'dirty_cat'})
dd=bench.melt(id_vars=['dataset','row','col'])
dd=dd[dd.dataset.str.contains('winlogs')]

dd.value=dd.value.astype('float')
dd.row=dd.row.astype('int')
dd=dd.rename(columns={'row':'row_count','value':'time (sec)'})
dd

dd=dd[dd.row_count<1000000]
dd=dd[dd.row_count>2000]


plt.figure(figsize=(8, 6))

import seaborn as sns

capsize = .1

ax = sns.barplot(data=dd,  x="row_count", y="time (sec)", hue="variable", capsize=capsize)

patches = ax.patches
lines_per_err = 1 if capsize is None else 3
ax.bar_label(ax.containers[0], fmt='%.1f')
ax.bar_label(ax.containers[1], fmt='%.1f')

for i, line in enumerate(ax.get_lines()):
    newcolor = patches[i // lines_per_err]#.get_facecolor()
    line.set_color('black')

fig = ax.get_figure()
fig.savefig('winlogs_'+str(gpu.strip())+'_bench_'+timestr+'.png')


In [ ]:
from google.colab import files
files.download('winlogs_'+str(gpu.strip())+'_bench_'+timestr+'.txt')
files.download('winlogs_'+str(gpu.strip())+'_bench_'+timestr+'.png')

In [ ]:
bench

In [ ]:
from importlib import reload
# import
reload(cu_cat)

# with more columns

In [ ]:

n=[2000,10000,50000,100000,500000]

import string, random
def id_generator(size=12, chars=string.ascii_uppercase + string.digits + ' '):
    return ''.join(random.choice(chars) for _ in range(size))

for ii,i in enumerate(n):
    winlogs=winlogsA.sample(i,replace=False)#.fillna(0)

    # print(i)
    # winlogs=winlogs[AA]
    winlogs=winlogs[['EventID','LogHost','UserName','Time','DomainName','LogonType','SubjectLogonID','Status','Destination','ServiceName']]#,i]]
    CC=winlogs.sample(20,replace=True,axis=1)

    winlogs=pd.DataFrame((np.apply_along_axis(np.random.permutation, 0, CC)))
    winlogs.columns=winlogs.columns.astype(str)

    winlogs['str1'] = np.array([id_generator(10) for i in range(i)])#.reshape(-1,2)
    words = [''.join(random.choice(string.ascii_uppercase + string.digits + ' ') for j in range(20)) for i in range(i)]
    winlogs['str2']=words

    g0 = graphistry.nodes((winlogs))
    t=time()
    # try:
    g00=g0.umap(engine='cuda',memoize=False)
    # g00=g0.umap(feature_engine='cu_cat',engine='cuml',memoize=False)
    t1=time()-t
    print('\n',t1,'\n')
    # except:
      # print('didnt run:',i,'/n')

    g1 = graphistry.nodes(winlogs)
    t=time()
    g11=g1.umap(feature_engine='dirty_cat',engine='cuml',memoize=False)
    t2=time()-t
    print('\n',t2,'\n')
    bench.loc[1+bench.shape[0]]=['winlogs',winlogs.shape[0],winlogs.shape[1],t1,t2]
    bench.to_csv('winlogs_'+str(gpu.strip())+'_wide_bench_'+timestr+'.txt',sep='\t',mode='a')
#